# Independence of Axioms

This notebook checks the environments labeled "Fact" in the paper "Characterizations of voting rules based on majority margins" by Yifeng Ding, Wesley H. Holliday, and Eric Pacuit. 

The main point is to supply concrete counterexamples showing that certain voting rules do not satisfy certain axioms.

In cases where we claim in the paper that certain voting rules *do* satisfy certain axioms, we include a search that turns up no counterexample. Of course such a failed search is not a proof, but proofs of the claims are themselves obvious.

In [1]:
import pref_voting
from pref_voting.generate_profiles import * 
from pref_voting.voting_methods import *
from pref_voting.rankings import *
from pref_voting.profiles_with_ties import ProfileWithTies
from pref_voting.iterative_methods import *
from pref_voting.variable_voter_axioms  import *
from pref_voting.invariance_axioms import *

from tqdm.notebook import tqdm

In [2]:
print(pref_voting.__version__)

1.14.30


## Fact 2.6

In [3]:
def opp_consecutive(r1, r2):
    """Returns True if the list r1 ranks some candidate immediately before r2 while r2 ranks the same candidate immediately before r1"""
    pos_in_r2 = {item: idx for idx, item in enumerate(r2)}
    return any(pos_in_r2[x] == pos_in_r2[y] + 1 for x, y in zip(r1, r1[1:]))

@vm(name="Borda or Plurality")
def borda_or_plurality(prof, curr_cands = None):
    """If there are two voters who rank two candidates consecutively but oppositely, use Borda. Otherwise, use Plurality"""
    if curr_cands is None:
        curr_cands = prof.candidates
    
    use_borda = False

    for r1 in prof.rankings:
        for r2 in prof.rankings:
            if opp_consecutive(r1, r2):
                use_borda = True
                break
            
    if use_borda:
        #print("Using Borda")
        return borda(prof)
    else:
        #print("Using Plurality")
        return plurality(prof)

In [4]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,11)

    if has_preferential_equality_violation(prof, borda_or_plurality, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

## Fact 2.8.1

In [5]:
@vm(name="Pareto")
def pareto_rule(prof, curr_cands = None):
    """Return the candidates who are not Pareto dominated in the profile prof."""

    if curr_cands is None:
        curr_cands = prof.candidates

    Pareto_dominated = list()

    for c in curr_cands:
        for d in curr_cands:
            if c != d:
                if prof.margin(c, d) == prof.num_voters:
                    Pareto_dominated.append(d)

    return sorted([c for c in prof.candidates if c not in Pareto_dominated])

In [6]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,11)

    if has_preferential_equality_violation(prof, pareto_rule, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

In [7]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,3)

    if has_neutral_reversal_violation(prof, pareto, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

Adding voters with rankings (0, 1, 2, 3) and (3, 2, 1, 0) changes the winners:

Original profile:
+---+---+---+
| 1 | 1 | 1 |
+---+---+---+
| 1 | 1 | 0 |
| 2 | 3 | 2 |
| 0 | 2 | 1 |
| 3 | 0 | 3 |
+---+---+---+
Pareto winners are {0, 1, 2}

Profile after adding reversal pair:
+---+---+---+---+---+
| 1 | 1 | 1 | 1 | 1 |
+---+---+---+---+---+
| 1 | 1 | 0 | 0 | 3 |
| 2 | 3 | 2 | 1 | 2 |
| 0 | 2 | 1 | 2 | 1 |
| 3 | 0 | 3 | 3 | 0 |
+---+---+---+---+---+
Pareto winners are {0, 1, 2, 3}


## Fact 2.8.2

In [8]:
def pos_neg(num_cands, rank):
    if rank == 1:
        return 1
    elif rank == num_cands:
        return -1
    else:
        return 0

positive_negative = create_scoring_method(score = lambda num_cands, rank : pos_neg(num_cands, rank), name = "Positive/Negative Voting")

In [9]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,11)

    if has_neutral_reversal_violation(prof, positive_negative, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

In [10]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,3)

    if has_preferential_equality_violation(prof, positive_negative, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

The original profile
+---+---+---+
| 1 | 1 | 1 |
+---+---+---+
| 1 | 2 | 3 |
| 3 | 1 | 2 |
| 0 | 0 | 1 |
| 2 | 3 | 0 |
+---+---+---+
Profile([[1, 3, 0, 2], [2, 1, 0, 3], [3, 2, 1, 0]], rcounts=[1, 1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3'})

The profile after swapping 0 and 1 in the rankings [(2, 1, 0, 3)]:
+---+---+---+
| 1 | 1 | 1 |
+---+---+---+
| 2 | 3 | 1 |
| 0 | 2 | 3 |
| 1 | 1 | 0 |
| 3 | 0 | 2 |
+---+---+---+
Profile([[2, 0, 1, 3], [3, 2, 1, 0], [1, 3, 0, 2]], rcounts=[1, 1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3'})
Positive/Negative Voting winner is {1}

The profile after swapping 0 and 1 in the rankings [(3, 2, 1, 0)]:
+---+---+---+
| 1 | 1 | 1 |
+---+---+---+
| 3 | 2 | 1 |
| 2 | 1 | 3 |
| 0 | 0 | 0 |
| 1 | 3 | 2 |
+---+---+---+
Profile([[3, 2, 0, 1], [2, 1, 0, 3], [1, 3, 0, 2]], rcounts=[1, 1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3'})
Positive/Negative Voting winners are {0, 1, 2, 3}


## Fact 3.4

In [11]:
@vm(name="All or Plurality")
def all_or_plurality(prof, curr_cands = None):

    if curr_cands is None:
        curr_cands = prof.candidates

    if isinstance(prof, Profile):
        
        last_place_disagreement = False

        for r1 in prof.rankings:
            for r2 in prof.rankings:
                if r1[-1] != r2[-1]:
                    last_place_disagreement = True
                    break
        
        if last_place_disagreement:
            return curr_cands
        else:   
            return plurality(prof)

    if isinstance(prof, ProfileWithTies):

        has_tie = False

        for r in prof.rankings:
            if r.to_linear == None:
                has_tie = True
                break

        if has_tie:
            print("has tie")
            return curr_cands
        
        last_place_disagreement = False

        for r1 in prof.rankings:
            for r2 in prof.rankings:
                if r1.last() != r2.last():
                    last_place_disagreement = True
                    break

        if last_place_disagreement:
            return curr_cands
        
        return plurality(prof)

In [12]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,11)
    prof = prof.to_profile_with_ties()

    if has_tiebreaking_compensation_violation(prof, all_or_plurality, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

In [13]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,3)
    prof = prof.to_profile_with_ties()

    if has_preferential_equality_violation(prof, all_or_plurality, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

The original profile
+---+---+---+
| 1 | 1 | 1 |
+---+---+---+
| 0 | 0 | 2 |
| 1 | 2 | 0 |
| 2 | 1 | 1 |
| 3 | 3 | 3 |
+---+---+---+
ProfileWithTies([{0: 0, 1: 1, 2: 2, 3: 3}, {0: 0, 2: 1, 1: 2, 3: 3}, {2: 0, 0: 1, 1: 2, 3: 3}], rcounts=[1, 1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3'})

The profile after swapping 0 and 1 in the rankings [<pref_voting.rankings.Ranking object at 0x177965ca0>]:
+---+---+---+
| 1 | 1 | 1 |
+---+---+---+
| 1 | 2 | 0 |
| 0 | 0 | 2 |
| 2 | 1 | 1 |
| 3 | 3 | 3 |
+---+---+---+
ProfileWithTies([{0: 1, 1: 0, 2: 2, 3: 3}, {2: 0, 0: 1, 1: 2, 3: 3}, {0: 0, 2: 1, 1: 2, 3: 3}], rcounts=[1, 1, 1], cmap={0: 0, 1: 1, 2: 2, 3: 3})
All or Plurality winners are {0, 1, 2}

The profile after swapping 0 and 1 in the rankings [<pref_voting.rankings.Ranking object at 0x177a89a30>]:
+---+---+---+
| 1 | 1 | 1 |
+---+---+---+
| 2 | 0 | 0 |
| 1 | 1 | 2 |
| 0 | 2 | 1 |
| 3 | 3 | 3 |
+---+---+---+
ProfileWithTies([{2: 0, 0: 2, 1: 1, 3: 3}, {0: 0, 1: 1, 2: 2, 3: 3}, {0: 0, 2: 1, 1: 2, 

## Fact 3.5.1

In [14]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,11)
    prof = prof.to_profile_with_ties()

    if has_tiebreaking_compensation_violation(prof, pareto_rule, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

## Fact 3.5.2

In [15]:
@vm(name="Minimax (winning votes)")
def minimax_winning_votes(prof, curr_cands = None):
    return minimax(prof, curr_cands = curr_cands, strength_function=prof.support)
a=0
b=1
c=2
cmap = {a: "a", b: "b", c: "c"}
prof  = ProfileWithTies([
    {b:1, a:2, c:2},
    {c:1, a:2, b:3},
    {a:1, b:2, c:3},
    ], 
    rcounts=[3, 4, 2],
    cmap=cmap)

has_tiebreaking_compensation_violation(prof, minimax_winning_votes, verbose = True)


+-----+---+---+
|  3  | 4 | 2 |
+-----+---+---+
|  b  | c | a |
| a c | a | b |
|     | b | c |
+-----+---+---+
ProfileWithTies([{1: 1, 0: 2, 2: 2}, {2: 1, 0: 2, 1: 3}, {0: 1, 1: 2, 2: 3}], rcounts=[3, 4, 2], cmap={0: 'a', 1: 'b', 2: 'c'})
Minimax (winning votes) winner is {a}

After breaking the tie between the candidates ['a', 'c'] in b ( a  c )  with ('a', 'c') and b ( a  c )  with ('c', 'a'): 

+---+---+-----+---+---+
| 1 | 1 |  1  | 4 | 2 |
+---+---+-----+---+---+
| 1 | 1 |  1  | 2 | 0 |
| 0 | 2 | 0 2 | 0 | 1 |
| 2 | 0 |     | 1 | 2 |
+---+---+-----+---+---+
ProfileWithTies([{1: 1, 0: 2, 2: 3}, {1: 1, 2: 2, 0: 3}, {1: 1, 0: 2, 2: 2}, {2: 1, 0: 2, 1: 3}, {2: 1, 0: 2, 1: 3}, {2: 1, 0: 2, 1: 3}, {2: 1, 0: 2, 1: 3}, {0: 1, 1: 2, 2: 3}, {0: 1, 1: 2, 2: 3}], rcounts=[1, 1, 1, 1, 1, 1, 1, 1, 1], cmap={0: 0, 1: 1, 2: 2})
Minimax (winning votes) winners are {0, 2}


True

In [16]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,11)

    if has_neutral_reversal_violation(prof, minimax_winning_votes, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

In [17]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,11)

    if has_preferential_equality_violation(prof, minimax_winning_votes, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

## Fact 3.7

In [18]:
@vm(name="BP or RP based on margins")
def bp_or_rp_margins(prof, curr_cands = None):

    max_margin = max([prof.margin(a,b) for a in prof.candidates for b in prof.candidates if a != b])

    if max_margin < 10:
        return beat_path(prof, curr_cands)
    
    else:
        return ranked_pairs(prof, curr_cands)

In [19]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,25)

    if has_homogeneity_violation(prof, bp_or_rp_margins, num_copies=2, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

Homogeneity Violation for BP or RP based on margins
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 |
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| 3 | 3 | 2 | 2 | 1 | 0 | 3 | 1 | 2 | 3 | 2 | 2 | 1 | 0 | 0 | 3 | 1 | 3 | 1 | 0 | 3 | 0 | 3 | 2 | 2 |
| 2 | 1 | 1 | 0 | 0 | 2 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 2 | 0 | 2 | 1 | 2 | 1 | 0 | 3 | 2 | 1 | 3 |
| 0 | 2 | 3 | 3 | 2 | 1 | 1 | 2 | 0 | 1 | 3 | 0 | 2 | 3 | 3 | 2 | 3 | 2 | 0 | 2 | 1 | 1 | 1 | 3 | 1 |
| 1 | 0 | 0 | 1 | 3 | 3 | 2 | 3 | 3 | 2 | 1 | 3 | 3 | 2 | 1 | 1 | 0 | 0 | 3 | 3 | 2 | 2 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
Profile([[3, 2, 0, 1], [3, 1, 2, 0], [2, 1, 3, 0], [2, 0, 3, 1], [1, 0, 2, 3], [0, 2, 1, 3], [3, 0, 1, 2], [1, 0, 2, 3], [2, 1, 0, 3

## Fact 3.10

In [20]:
@vm(name="BP or RP based on parity of voters")
def bp_or_rp_parity(prof, curr_cands = None):

    if curr_cands is None:
        curr_cands = prof.candidates

    if prof.num_voters % 2 == 0:
        return beat_path(prof, curr_cands)
    else:
        return ranked_pairs(prof, curr_cands)

In [21]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,5)

    if has_homogeneity_violation(prof, bp_or_rp_parity, num_copies=2, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

Homogeneity Violation for BP or RP based on parity of voters
+---+---+---+---+---+
| 1 | 1 | 1 | 1 | 1 |
+---+---+---+---+---+
| 2 | 0 | 2 | 1 | 3 |
| 3 | 1 | 1 | 3 | 0 |
| 0 | 3 | 3 | 2 | 1 |
| 1 | 2 | 0 | 0 | 2 |
+---+---+---+---+---+
Profile([[2, 3, 0, 1], [0, 1, 3, 2], [2, 1, 3, 0], [1, 3, 2, 0], [3, 0, 1, 2]], rcounts=[1, 1, 1, 1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3'})
BP or RP based on parity of voters winners are {1, 3}

+---+---+---+---+---+
| 2 | 2 | 2 | 2 | 2 |
+---+---+---+---+---+
| 2 | 0 | 2 | 1 | 3 |
| 3 | 1 | 1 | 3 | 0 |
| 0 | 3 | 3 | 2 | 1 |
| 1 | 2 | 0 | 0 | 2 |
+---+---+---+---+---+
Profile([[2, 3, 0, 1], [0, 1, 3, 2], [2, 1, 3, 0], [1, 3, 2, 0], [3, 0, 1, 2]], rcounts=[2, 2, 2, 2, 2], cmap={0: '0', 1: '1', 2: '2', 3: '3'})
BP or RP based on parity of voters winners are {1, 2, 3}


In [22]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,3)

    if has_neutral_reversal_violation(prof, bp_or_rp_parity, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

## Footnote 9

In [92]:
# Given $Y\subsetneq X$ and rankings $P$ and $P'$ of $X$, say that $P$ and $P'$ \textit{agree outside $Y$}, if (a) for all $x\in X\setminus Y$, either $xPy$ for all $y\in Y$ or $yPx$ for all $y\in Y$, and either $xP'y$ for all $y\in Y$ or $yP'x$ for all $y\in Y$,  and (b) $P\setminus Y^2 = P'\setminus Y^2$.

def agree_outside_Y(X, Y, P, P2):
    # first check condition (a) for P
    for x in X:
        if x in Y:
            continue
        
        above_all_ys = True

        for y in Y:
            if not P[x] < P[y]:
                above_all_ys = False
                break
        
        below_all_ys = True

        for y in Y:
            if not P[y] < P[x]:
                below_all_ys = False
                break
                
        if not (above_all_ys or below_all_ys):
            return False

    # now check condition (a) for P'
    for x in X:
        if x in Y:
            continue
        
        above_all_ys = True

        for y in Y:
            if not P2[x] < P2[y]:
                above_all_ys = False
                break
        
        below_all_ys = True

        for y in Y:
            if not P2[y] < P2[x]:
                below_all_ys = False
                break
                
        if not (above_all_ys or below_all_ys):
            return False

    # now check condition (b)
    for x in X:
        for z in X:
            if x in Y and z in Y:
                continue
            
            # check whether P and P' agree on the relative order of x and z
            if not P[x] < P[z] == P2[x] < P2[z]:
                return False

    return True

def agree_outside_some_Y(X, P, P2):
    for Y in powerset(X):
        if len(Y) != len(X) and agree_outside_Y(X, Y, P, P2):
            return True
    return False

In [93]:
@vm(name="Footnote 9 Rule")
def footnote_9_rule(prof, curr_cands = None):

    curr_cands = prof.candidates
    
    apply_first_case = False

    rs = prof.rankings

    for r in rs:
        r_reversed = list(reversed(r))
        for r2 in rs:
            if agree_outside_some_Y(prof.candidates, r, r2) or r2 == r_reversed:
                apply_first_case = True
                break

    if apply_first_case:
        return curr_cands
    
    else:
        return plurality(prof)

In [94]:
fn_9_prof = Profile([[0,1,2,3,4], [3,0,1,4,2]])
has_preferential_equality_violation(fn_9_prof, footnote_9_rule, verbose = True)

The original profile
+---+---+
| 1 | 1 |
+---+---+
| 0 | 3 |
| 1 | 0 |
| 2 | 1 |
| 3 | 4 |
| 4 | 2 |
+---+---+
Profile([[0, 1, 2, 3, 4], [3, 0, 1, 4, 2]], rcounts=[1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3', 4: '4'})

The profile after swapping 0 and 1 in the rankings [(0, 1, 2, 3, 4)]:
+---+---+
| 1 | 1 |
+---+---+
| 1 | 3 |
| 0 | 0 |
| 2 | 1 |
| 3 | 4 |
| 4 | 2 |
+---+---+
Profile([[1, 0, 2, 3, 4], [3, 0, 1, 4, 2]], rcounts=[1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3', 4: '4'})
Footnote 9 Rule winners are {1, 3}

The profile after swapping 0 and 1 in the rankings [(3, 0, 1, 4, 2)]:
+---+---+
| 1 | 1 |
+---+---+
| 3 | 0 |
| 1 | 1 |
| 0 | 2 |
| 4 | 3 |
| 2 | 4 |
+---+---+
Profile([[3, 1, 0, 4, 2], [0, 1, 2, 3, 4]], rcounts=[1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3', 4: '4'})
Footnote 9 Rule winners are {0, 3}


True

## Fact 4.2

In [23]:
@vm(name="Borda for SWOs")
def borda_swo(prof, curr_cands = None):

    borda_score = {c:0 for c in prof.candidates}

    for c in prof.candidates:
        for r in prof.rankings:
            # add to the Borda score for c the number of candidates ranked below it
            if isinstance(prof, Profile):
                borda_score[c] += sum([1 for d in prof.candidates if r.index(c) < r.index(d)])
            if isinstance(prof, ProfileWithTies):
                borda_score[c] += sum([1 for d in prof.candidates if r.strict_pref(c, d)])

    max_borda_score = max(borda_score.values())

    return sorted([c for c in prof.candidates if borda_score[c] == max_borda_score])

In [24]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,3)

    if has_nonlinear_neutral_reversal_violation(prof, borda_swo, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

The original profile
+---+---+---+
| 1 | 1 | 1 |
+---+---+---+
| 0 | 1 | 3 |
| 1 | 0 | 0 |
| 3 | 2 | 1 |
| 2 | 3 | 2 |
+---+---+---+
Profile([[0, 1, 3, 2], [1, 0, 2, 3], [3, 0, 1, 2]], rcounts=[1, 1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3'})
Borda for SWOs winner is {0}

The profile after adding 1 2 ( 0  3 )  and its reverse ( 0  3 ) 2 1 :
+---+---+---+-----+-----+
| 1 | 1 | 1 |  1  |  1  |
+---+---+---+-----+-----+
| 0 | 1 | 3 |  1  | 0 3 |
| 1 | 0 | 0 |  2  |  2  |
| 3 | 2 | 1 | 0 3 |  1  |
| 2 | 3 | 2 |     |     |
+---+---+---+-----+-----+
ProfileWithTies([{0: 1, 1: 2, 3: 3, 2: 4}, {1: 1, 0: 2, 2: 3, 3: 4}, {3: 1, 0: 2, 1: 3, 2: 4}, {1: 1, 2: 2, 0: 3, 3: 3}, {1: 3, 2: 2, 0: 1, 3: 1}], rcounts=[1, 1, 1, 1, 1], cmap={0: 0, 1: 1, 2: 2, 3: 3})
Borda for SWOs winners are {0, 1}


In [25]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,11)

    if has_neutral_reversal_violation(prof, borda_swo, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

## Fact 4.5.1

In [26]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,11)

    if has_neutral_indifference_violation(prof, minimax_winning_votes, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

In [27]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,3)

    if has_nonlinear_neutral_reversal_violation(prof, minimax_winning_votes, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

The original profile
+---+---+---+
| 1 | 1 | 1 |
+---+---+---+
| 3 | 1 | 0 |
| 2 | 2 | 1 |
| 0 | 0 | 2 |
| 1 | 3 | 3 |
+---+---+---+
Profile([[3, 2, 0, 1], [1, 2, 0, 3], [0, 1, 2, 3]], rcounts=[1, 1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3'})
Minimax (winning votes) winners are {0, 1, 2, 3}

The profile after adding 0 3 ( 1  2 )  and its reverse ( 1  2 ) 3 0 :
+---+---+---+-----+-----+
| 1 | 1 | 1 |  1  |  1  |
+---+---+---+-----+-----+
| 3 | 1 | 0 |  0  | 1 2 |
| 2 | 2 | 1 |  3  |  3  |
| 0 | 0 | 2 | 1 2 |  0  |
| 1 | 3 | 3 |     |     |
+---+---+---+-----+-----+
ProfileWithTies([{3: 1, 2: 2, 0: 3, 1: 4}, {1: 1, 2: 2, 0: 3, 3: 4}, {0: 1, 1: 2, 2: 3, 3: 4}, {0: 1, 3: 2, 1: 3, 2: 3}, {0: 3, 3: 2, 1: 1, 2: 1}], rcounts=[1, 1, 1, 1, 1], cmap={0: 0, 1: 1, 2: 2, 3: 3})
Minimax (winning votes) winner is {2}


## Fact 4.5.2

In [28]:
for n in tqdm(range(10_000)):
    prof = generate_profile(4,5)

    if has_neutral_indifference_violation(prof, bp_or_rp_parity, verbose = True):
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

The original profile
+---+---+---+---+
| 1 | 2 | 1 | 1 |
+---+---+---+---+
| 3 | 2 | 0 | 1 |
| 0 | 3 | 2 | 3 |
| 1 | 1 | 1 | 0 |
| 2 | 0 | 3 | 2 |
+---+---+---+---+
Profile([[3, 0, 1, 2], [2, 3, 1, 0], [0, 2, 1, 3], [2, 3, 1, 0], [1, 3, 0, 2]], rcounts=[1, 1, 1, 1, 1], cmap={0: '0', 1: '1', 2: '2', 3: '3'})
BP or RP based on parity of voters winners are {2, 3}

The profile after adding a tie ranking:
+---+---+---+---+---------+
| 1 | 2 | 1 | 1 |    1    |
+---+---+---+---+---------+
| 3 | 2 | 0 | 1 | 0 1 2 3 |
| 0 | 3 | 2 | 3 |         |
| 1 | 1 | 1 | 0 |         |
| 2 | 0 | 3 | 2 |         |
+---+---+---+---+---------+
ProfileWithTies([{3: 1, 0: 2, 1: 3, 2: 4}, {2: 1, 3: 2, 1: 3, 0: 4}, {0: 1, 2: 2, 1: 3, 3: 4}, {2: 1, 3: 2, 1: 3, 0: 4}, {1: 1, 3: 2, 0: 3, 2: 4}, {0: 0, 1: 0, 2: 0, 3: 0}], rcounts=[1, 1, 1, 1, 1, 1], cmap={0: 0, 1: 1, 2: 2, 3: 3})
BP or RP based on parity of voters winners are {1, 2, 3}


## Fact 4.8

In [29]:
## To be added